Import allowed guesses and precomputed colouring data


In [1]:
with open("../data/allowed_guesses.txt", "r") as fhand:
    actions = [word.strip() for word in fhand]
    actions = tuple(actions)

import json
with open("../data/precompute.json", "r") as file:
    colouring_data = json.load(file)

In [2]:
def get_transition_info(state, action):
    new_state = {}
    for word in state:
        colouring = colouring_data[action][word] 
        if colouring not in new_state:
            new_state[colouring] = [word]
        else:
            new_state[colouring].append(word)
    transition_info = {tuple(states): (len(states) / len(state)) for states in new_state.values()}
    return transition_info

In [3]:
def value_function(t, state, actions, v_mem={}):
    if (t, state) in v_mem:
        return v_mem[(t, state)]
    
    if t >= 6 or (t == 5 and len(state) > 1):
        return (float("inf"), None)
    if t == 5 or len(state) == 1:
        return (1, (state))
    elif len(state) == 2:
        return (1.5, (state)[0])

    state_value = float('inf')
    best_word = None
    
    for action in actions:
        if t == 1:
            print(f"Searching for best word: {actions.index(action) / len(actions): .3%}, Current word = {action}")
    
        
        transition_info = get_transition_info(state, action)
        temp = 1
        
        if len(transition_info.keys()) == 1 and tuple(transition_info.keys())[0] == state:
            continue
        
        temp += (2 * len(state) - 1) / len(state)

        for st1 in transition_info.keys():
            
            if temp >= state_value:
                break
            elif st1 == action:
                continue
            
            res, _ = value_function(t + 1, state=st1, actions=actions)
            temp += transition_info[st1] * res

        if temp < state_value:
            state_value = temp
            best_word = action
    
    v_mem[(t, state)] = (state_value, best_word)
    return state_value, best_word

In [4]:
import os
import pickle
def load_v_mem(file_path = "v_mem.pkl") -> dict:
    if os.path.exists(file_path):
        with open(file_path, 'rb') as file:
            v_mem = pickle.load(file)
        return v_mem
    else:
        print(f"The file {file_path} does not exist.")
        return {}
    


In [5]:
def execute_and_save(t, state, actions, v_mem):
    result = value_function(t, state, actions, v_mem)
    
    with open('v_mem.pkl', 'wb') as file:
        pickle.dump(v_mem, file)
    
    return result

In [6]:
v_mem = load_v_mem("v_mem.pkl")

The file v_mem.pkl does not exist.


In [7]:
def value_function_wrapper(args):
    t, state, actions = args
    print(f"Starting process for t = {t}, state = {state} actions = {actions}")
    return value_function(t, state, actions)

In [18]:
import multiprocessing as mp


state = ("delay", "fella", "relax", "relay")
actions = state + actions

# Get the number of available CPUs, with a max of 6
max_cpu = 1
cpu_count = min(max_cpu, mp.cpu_count())




# Split the actions into cpu_count chunks
num_chunks = cpu_count 
chunk_size = len(actions) / num_chunks 
indices = [int(i * chunk_size) for i in range(num_chunks + 1)] 
chunks = [actions[indices[i]:indices[i + 1]] for i in range(len(indices) - 1)]


# Prepare the arguments for each process
tasks = [(0, state, chunk) for chunk in chunks]

# Create a pool of processes
with mp.Pool(cpu_count) as pool:
    results = pool.map(value_function_wrapper, tasks)


